In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset, DatasetDict
import torch.utils.data as data
from transformers import AutoTokenizer
import math
import copy
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
import time
from tqdm import tqdm
qa_dataset = load_dataset("maywell/ko_wikidata_QA", cache_dir="../../../data/jeongseokoh/dataset/")
tokenizer_ko = AutoTokenizer.from_pretrained("klue/bert-base")
# Train 데이터셋을 랜덤하게 섞습니다.
shuffled_dataset = qa_dataset['train'].shuffle(seed=42)

# 30,000개를 train용, 10,000개를 validation용으로 분리
train_dataset = shuffled_dataset.select(range(30000))
validation_dataset = shuffled_dataset.select(range(30000, 40000))


def ko_tokenize_function(examples):
    # 영어(en)와 독일어(de) 텍스트에 대한 토크나이징 수행
    tokenized_inputs = tokenizer_ko(examples["instruction"], padding="max_length", truncation=True, max_length=256)
    # 타겟 토크나이저로 영어 텍스트 토크나이징
    with tokenizer_ko.as_target_tokenizer():
        tokenized_targets = tokenizer_ko(examples["output"], padding="max_length", truncation=True, max_length=256)
    
    # 모델 입력을 위한 tokenized_inputs 반환 및 레이블(labels)로 tokenized_targets["input_ids"] 설정
    tokenized_inputs["labels"] = tokenized_targets["input_ids"]
    return tokenized_inputs
def collate_fn(batch):
    input_ids = torch.tensor([item['input_ids'] for item in batch])
    labels = torch.tensor([item['labels'] for item in batch])
    return {"input_ids": input_ids, "labels": labels}
# 데이터셋에 토큰화 함수 적용
tokenized_datasets = shuffled_dataset.map(ko_tokenize_function, batched=True)
val_tokenized_datasets = validation_dataset.map(ko_tokenize_function, batched=True)

# DataLoader 재정의
dataloader = DataLoader(tokenized_datasets, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_tokenized_datasets, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(val_tokenized_datasets, batch_size=1, shuffle=True, collate_fn=collate_fn)

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        #print(f"d_model : {d_model}")
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        #print(f"Q: {Q.shape}")

        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        #print(Q.shape)
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        #print(f"Output: {output.shape}")
        return output
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))
    
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        #print((x + self.pe[:, :x.size(1)]).shape)
        return x + self.pe[:, :x.size(1)]
## 1. CustomWeightLayer를 expand와 shrink로 나눈다. 
## 2-1. 768차원으로 늘려서 Encoder의 Self-attention에 넣는다.
## 2-2. 768차원으로 늘려서 바로 decoder의 Cross-attention에 넣는다. 


class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout, tokens):
        super(EncoderLayer, self).__init__()
        self.tokens = tokens
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):

        our_output = self.self_attn(x, x, x, mask)
        
        x = self.norm1(x + self.dropout(our_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout, tokens):
        super(DecoderLayer, self).__init__()
        self.tokens = tokens
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout, max_seq_length) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout, max_seq_length) for _ in range(num_layers)])
        
        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2).to(device)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3).to(device)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(device)
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        seq_length = tgt.size(1)
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))
        
        enc_output = src_embedded
        for i, enc_layer in enumerate(self.encoder_layers):
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [3]:
import wandb
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
src_vocab_size = tokenizer_ko.vocab_size
tgt_vocab_size = tokenizer_ko.vocab_size
d_model = 768  # BERT의 경우
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 256
dropout = 0.1
wandb.init(
        # set the wandb project where this run will be logged
        project="Sparse_plane",
        name="Base_ko",
        # track hyperparameters and run metadata
        config={
            "learning_rate": 0.01,
            "architecture": "Scratch",
            "dataset": "de,en",
            "epochs": 10,
            }
    )
# 모델 초기화
transformer = Transformer(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_layers=num_layers,
    d_ff=d_ff,
    max_seq_length=max_seq_length,
    dropout=dropout
).to(device)
#model_state_dict = torch.load('../../../data/jeongseokoh/model/base_noise_de_en.pth')
#transformer.load_state_dict(model_state_dict)
# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer_ko.pad_token_id)
optimizer = optim.AdamW(transformer.parameters(), lr=0.0001, weight_decay=0.01)

# 모델 학습
transformer.train()


start_time = time.time()

for epoch in range(1, 11):  # 에폭 수 조정 가능
    total_loss = 0
    total_var = 0
    for batch in tqdm(dataloader, desc="Train Iteration"):
        src = batch['input_ids'].to(device)  # 장치로 이동
        tgt = batch['labels'].to(device)     # 장치로 이동

        optimizer.zero_grad()
        
        output = transformer(src, tgt[:, :-1])  # [32, 128, vocab_size], [32]
        
        # 교차 엔트로피 손실 계산
        # 교차 엔트로피 손실 계산
        loss = criterion(output.reshape(-1, tgt_vocab_size), tgt[:, 1:].reshape(-1))

        loss.backward()
        optimizer.step()
        wandb.log({"Loss": loss.item()})
        total_loss += loss.item()
        #total_var += variances.mean()
        
    avg_loss = total_loss / len(dataloader)
    #avg_var = total_var / len(dataloader)
    print(f"Epoch: {epoch}, Avg Train Loss: {avg_loss:.4f}")


    checkpoint = {
        'epoch': epoch,
        'model_state_dict': transformer.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avg_loss,
        # You can add more components here as needed.
    }
    torch.save(checkpoint, f'../../parameters/scratch/base_attention_ko_wiki_checkpoint_epoch_{epoch}.pth')
    total_loss = 0
    for batch in tqdm(val_dataloader, desc="Val Iteration"):
        src = batch['input_ids'].to(device)  # 장치로 이동
        tgt = batch['labels'].to(device)     # 장치로 이동

        with torch.no_grad():
            output = transformer(src, tgt[:, :-1])
            loss = criterion(output.reshape(-1, tgt_vocab_size), tgt[:, 1:].reshape(-1))  
        total_loss += loss.item()

    avg_loss = total_loss / len(val_dataloader)
    print(f"Epoch: {epoch}, Avg Validation Loss: {avg_loss:.4f}")
    wandb.log({"Val_Loss": avg_loss})  
    
end_time = time.time()
print(f"총 시간: {end_time - start_time} sec")
torch.save(transformer.state_dict(), '../../parameters/scratch/base_attention_ko_wiki.pth')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: luke0112. Use `wandb login --relogin` to force relogin


In [ ]:
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
src_vocab_size = tokenizer_ko.vocab_size
tgt_vocab_size = tokenizer_ko.vocab_size
d_model = 768  # BERT의 경우
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 128
dropout = 0.1

# 모델 초기화
transformer = Transformer(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_layers=num_layers,
    d_ff=d_ff,
    max_seq_length=max_seq_length,
    dropout=dropout
).to(device)
model_state_dict = torch.load('../../parameters/scratch/base_noise_de_en_checkpoint_epoch_8.pth')
transformer.load_state_dict(model_state_dict['model_state_dict'])
# 모델 초기화
#transformer2 = Transformer(
#    src_vocab_size=src_vocab_size,
#    tgt_vocab_size=tgt_vocab_size,
#    d_model=d_model,
#    num_heads=num_heads,
#    num_layers=num_layers,
#    d_ff=d_ff,
#    max_seq_length=max_seq_length,
#    dropout=dropout
#).to(device)
#model_state_dict = torch.load('../../parameters/scratch/base_de_en_checkpoint_epoch_7.pth')
#transformer2.load_state_dict(model_state_dict['model_state_dict'])
#criterion = nn.CrossEntropyLoss(ignore_index=tokenizer_tgt.pad_token_id)
def predict(model, input_sequence, max_length=15, SOS_token=101, EOS_token=102):
    model.eval()
    # Ensure input_sequence is a tensor and reshape if necessary
    if not isinstance(input_sequence, torch.Tensor):
        input_sequence = torch.tensor(input_sequence, dtype=torch.long)
    input_sequence = input_sequence.unsqueeze(0)  # Add batch dimension if missing
    input_sequence = input_sequence.to(device)  # Move to the same device as the model

    y_input = torch.tensor([[SOS_token]], dtype=torch.long, device=device)
    with torch.no_grad():
        for _ in range(max_length):
            pred, variances = model(input_sequence, y_input)
            next_item = pred[:, -1, :].topk(1)[1].view(-1).item()  # Get the last word's top prediction
            next_item = torch.tensor([[next_item]], device=device)
            #print(next_item)
            y_input = torch.cat((y_input, next_item), dim=1)
            if next_item.view(-1).item() == EOS_token:
                break

    return y_input.view(-1).tolist(), variances
 
val_dataloader.dataset['labels'][1]
# Here we test some examples to observe how the model predicts

examples2 = test_dataloader.dataset['input_ids'][:10]
 
for idx, example in enumerate(examples2):
    result, variances = predict(transformer, example)
    result2, variances2 = predict(transformer2, example)
    print(f"Example {idx}")
    print(f"Input: {example[1:-1]}")
    print(f"GoldenAnswer: {test_dataloader.dataset['en'][idx]}")
    print(f"SparseAnswer: {tokenizer_ko.decode(result[1:])}")
    print(f"Base  Answer: {tokenizer_ko.decode(result2[1:])}")
    print()
